<a href="https://colab.research.google.com/github/shahkarKhan24/Natural-Language-Inference-NLI/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate -U
!pip install datasets

   ---------------------------------------- 0.0/309.4 kB ? eta -:--:--
   ------------------------------ --------- 235.5/309.4 kB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 309.4/309.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from typing import Dict
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, load_metric
from scipy.special import softmax
from sklearn.metrics import roc_auc_score, hamming_loss
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    DistilBertForSequenceClassification,
    DataCollatorWithPadding,
    AutoTokenizer,
    EvalPrediction,
    DistilBertConfig,
    Trainer,
    EarlyStoppingCallback,
    TrainingArguments,
    set_seed,
)

c:\Users\shahk\AppData\Local\Programs\Python\Python39\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Load the dataset
dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli")
# View the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 51086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2287
    })
})


In [ ]:
# Access the training data
train_data = dataset['train']
# Print the first example
print(train_data[0])

{'id': '150448', 'premise': "Roman Atwood . He is best known for his vlogs , where he posts updates about his life on a daily basis . His vlogging channel , `` RomanAtwoodVlogs '' , has a total of 3.3 billion views and 11.9 million subscribers . He also has another YouTube channel called `` RomanAtwood '' , where he posts pranks .", 'hypothesis': 'Roman Atwood is a content creator.', 'label': 'ENTAILMENT', 'wsd': {'premise': [{'index': 0, 'text': 'Roman', 'pos': 'ADJ', 'lemma': 'roman', 'bnSynsetId': 'bn:00109913a', 'wnSynsetOffset': '2921569a', 'nltkSynset': 'roman.a.01'}, {'index': 1, 'text': 'Atwood', 'pos': 'PROPN', 'lemma': 'Atwood', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 2, 'text': '.', 'pos': 'PUNCT', 'lemma': '.', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 3, 'text': 'He', 'pos': 'PRON', 'lemma': 'he', 'bnSynsetId': 'O', 'wnSynsetOffset': 'O', 'nltkSynset': 'O'}, {'index': 4, 'text': 'is', 'pos': 'AUX', 'lemma': 'be'

In [ ]:
# Load and preprocess the adversarial test set
adversarial_test_dataset = load_dataset("iperbole/adversarial_fever_nli")
# View the dataset
print(adversarial_test_dataset)

DatasetDict({
    test: Dataset({
        features: ['part', 'cid', 'premise', 'hypothesis', 'label'],
        num_rows: 337
    })
})


In [ ]:
# Metrics
# Load metrics
load_accuracy = load_metric("accuracy",trust_remote_code=True)
load_precision = load_metric("precision",trust_remote_code=True)
load_recall = load_metric("recall",trust_remote_code=True)
load_f1 = load_metric("f1",trust_remote_code=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    precision = load_precision.compute(predictions=predictions, references=labels, average='weighted')["precision"]
    recall = load_recall.compute(predictions=predictions, references=labels, average='weighted')["recall"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average='weighted')["f1"]
    # Convert logits to probabilities
    probabilities = softmax(logits, axis=1)

    # Compute additional metrics
    roc_auc = roc_auc_score(labels, probabilities, multi_class='ovr')
    hamming = hamming_loss(labels, predictions)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "hamming_loss": hamming,
    }


C:\Users\shahk\AppData\Local\Temp\ipykernel_9972\2462315092.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy",trust_remote_code=True)


In [ ]:
torch.cuda.empty_cache()
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Roberta model
Training model # 1

In [ ]:
language_model_name = "roberta-base"
# this GPU should be enough for this task to handle 32 samples per batch
batch_size = 16
# optim
learning_rate = 1e-4
weight_decay = 0.001 # we could use e.g. 0.01 in case of very low and very high amount of data for regularization
# training
epochs = 1
device = "cuda" if torch.cuda.is_available() else "cpu"
set_seed(42)

## Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(language_model_name,
                                                                   ignore_mismatched_sizes=True,
                                                                   output_attentions=False, output_hidden_states=False,
                                                                   num_labels=3) # number of the classes

tokenizer = AutoTokenizer.from_pretrained(language_model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def encode_labels(example):
    label_map = {"ENTAILMENT": 0, "CONTRADICTION": 1, "NEUTRAL": 2}
    example["label"] = label_map[example["label"]]
    return example

def preprocess_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding='max_length', max_length=128)

# def preprocess_function(examples):
#     inputs = ["[CLS] " + premise + " [SEP] " + hypothesis for premise, hypothesis in zip(examples['premise'], examples['hypothesis'])]
#     return tokenizer(inputs, truncation=True, padding='max_length', max_length=128, add_special_tokens=False)

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.map(encode_labels)

Map:   0%|          | 0/2288 [00:00<?, ? examples/s]

Map:   0%|          | 0/51086 [00:00<?, ? examples/s]

Map:   0%|          | 0/2288 [00:00<?, ? examples/s]

Map:   0%|          | 0/2287 [00:00<?, ? examples/s]

In [ ]:

training_args = TrainingArguments(
    output_dir="./results",                    # output directory [Mandatory]
    num_train_epochs=1,                      # total number of training epochs
    per_device_train_batch_size=batch_size,       # batch size per device during training
    warmup_steps=500,                             # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                    # strength of weight decay
    save_strategy="no",
    learning_rate=learning_rate                   # learning rate
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shahkarkhan24 (shahkar). Use `wandb login --relogin` to force relogin


  0%|          | 0/3193 [00:00<?, ?it/s]

{'loss': 0.7615, 'grad_norm': 21.145954132080078, 'learning_rate': 0.0001, 'epoch': 0.16}
{'loss': 0.6901, 'grad_norm': 10.859151840209961, 'learning_rate': 8.143334571110286e-05, 'epoch': 0.31}
{'loss': 0.6637, 'grad_norm': 4.032456398010254, 'learning_rate': 6.286669142220572e-05, 'epoch': 0.47}
{'loss': 0.5839, 'grad_norm': 7.356015205383301, 'learning_rate': 4.430003713330858e-05, 'epoch': 0.63}
{'loss': 0.5216, 'grad_norm': 7.082505702972412, 'learning_rate': 2.5733382844411435e-05, 'epoch': 0.78}
{'loss': 0.4884, 'grad_norm': 5.406497001647949, 'learning_rate': 7.166728555514296e-06, 'epoch': 0.94}
{'train_runtime': 1707.7554, 'train_samples_per_second': 29.914, 'train_steps_per_second': 1.87, 'train_loss': 0.6079318712319919, 'epoch': 1.0}


TrainOutput(global_step=3193, training_loss=0.6079318712319919, metrics={'train_runtime': 1707.7554, 'train_samples_per_second': 29.914, 'train_steps_per_second': 1.87, 'total_flos': 3360353014513152.0, 'train_loss': 0.6079318712319919, 'epoch': 1.0})

Evaluaating roberta model on standard test dataset

In [ ]:
results_standard = trainer.evaluate(tokenized_datasets['test'])
print(f"Standard Test Set Results: {results_standard}")

  0%|          | 0/286 [00:00<?, ?it/s]

Standard Test Set Results: {'eval_loss': 0.8490660190582275, 'eval_accuracy': 0.6943594228246611, 'eval_precision': 0.6879176651634599, 'eval_recall': 0.6943594228246611, 'eval_f1': 0.6818675986758914, 'eval_roc_auc': 0.843863808216641, 'eval_hamming_loss': 0.3056405771753389, 'eval_runtime': 25.6202, 'eval_samples_per_second': 89.266, 'eval_steps_per_second': 11.163, 'epoch': 1.0}


Evaluaating roberta model on adverserial testset

In [ ]:
adversarial_tokenized = adversarial_test_dataset.map(preprocess_function, batched=True)
adversarial_tokenized = adversarial_tokenized.map(encode_labels)
# Evaluate on the adversarial test set
results_adversarial = trainer.evaluate(adversarial_tokenized)
print(f"Adversarial Test Set Results: {results_adversarial}")

  0%|          | 0/43 [00:00<?, ?it/s]

Adversarial Test Set Results: {'eval_test_loss': 1.3014122247695923, 'eval_test_accuracy': 0.5103857566765578, 'eval_test_precision': 0.516193648089156, 'eval_test_recall': 0.5103857566765578, 'eval_test_f1': 0.5101830483923316, 'eval_test_roc_auc': 0.6715665865686232, 'eval_test_hamming_loss': 0.4896142433234421, 'eval_test_runtime': 3.9488, 'eval_test_samples_per_second': 85.342, 'eval_test_steps_per_second': 10.889, 'epoch': 1.0}


# Distilbert_uncased
Training model # 2

In [ ]:
language_model_name = "distilbert-base-uncased"
# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(language_model_name,
                                                                   ignore_mismatched_sizes=True,
                                                                   output_attentions=False, output_hidden_states=False,
                                                                   num_labels=3) # number of the classes

tokenizer = AutoTokenizer.from_pretrained(language_model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def encode_labels(example):
    label_map = {"ENTAILMENT": 0, "CONTRADICTION": 1, "NEUTRAL": 2}
    example["label"] = label_map[example["label"]]
    return example

def preprocess_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding='max_length', max_length=128)

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.map(encode_labels)


Training arguments remains the same

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)
trainer.train()

  0%|          | 0/3193 [00:00<?, ?it/s]

{'loss': 0.6934, 'grad_norm': 7.663567543029785, 'learning_rate': 0.0001, 'epoch': 0.16}
{'loss': 0.5563, 'grad_norm': 4.596882343292236, 'learning_rate': 8.143334571110286e-05, 'epoch': 0.31}
{'loss': 0.5004, 'grad_norm': 2.569326400756836, 'learning_rate': 6.286669142220572e-05, 'epoch': 0.47}
{'loss': 0.4751, 'grad_norm': 7.720330238342285, 'learning_rate': 4.430003713330858e-05, 'epoch': 0.63}
{'loss': 0.4518, 'grad_norm': 2.6637659072875977, 'learning_rate': 2.5733382844411435e-05, 'epoch': 0.78}
{'loss': 0.4457, 'grad_norm': 5.572751522064209, 'learning_rate': 7.166728555514296e-06, 'epoch': 0.94}
{'train_runtime': 870.0749, 'train_samples_per_second': 58.714, 'train_steps_per_second': 3.67, 'train_loss': 0.5137281313309877, 'epoch': 1.0}


TrainOutput(global_step=3193, training_loss=0.5137281313309877, metrics={'train_runtime': 870.0749, 'train_samples_per_second': 58.714, 'train_steps_per_second': 3.67, 'total_flos': 1691837552918016.0, 'train_loss': 0.5137281313309877, 'epoch': 1.0})

Evaluating DistilBert model on standard test dataset

In [ ]:
results_standard = trainer.evaluate(tokenized_datasets['test'])
print(f"Standard Test Set Results: {results_standard}")

  0%|          | 0/286 [00:00<?, ?it/s]

Standard Test Set Results: {'eval_loss': 0.7932063937187195, 'eval_accuracy': 0.7022299956274596, 'eval_precision': 0.7012149224357939, 'eval_recall': 0.7022299956274596, 'eval_f1': 0.6923819271909458, 'eval_roc_auc': 0.8546023759794337, 'eval_hamming_loss': 0.2977700043725404, 'eval_runtime': 13.8056, 'eval_samples_per_second': 165.657, 'eval_steps_per_second': 20.716, 'epoch': 1.0}


Evaluating DistilBert model on adverserial testset

In [ ]:
adversarial_tokenized = adversarial_test_dataset.map(preprocess_function, batched=True)
adversarial_tokenized = adversarial_tokenized.map(encode_labels)
# Evaluate on the adversarial test set
results_adversarial = trainer.evaluate(adversarial_tokenized)
print(f"Adversarial Test Set Results: {results_adversarial}")

  0%|          | 0/43 [00:00<?, ?it/s]

Adversarial Test Set Results: {'eval_test_loss': 1.3519256114959717, 'eval_test_accuracy': 0.516320474777448, 'eval_test_precision': 0.5411160930290758, 'eval_test_recall': 0.516320474777448, 'eval_test_f1': 0.518962524502693, 'eval_test_roc_auc': 0.6570465309504344, 'eval_test_hamming_loss': 0.4836795252225519, 'eval_test_runtime': 2.2066, 'eval_test_samples_per_second': 152.724, 'eval_test_steps_per_second': 19.487, 'epoch': 1.0}


Both model was giving almost same result on standard test set and adversarial set but “roberta ” model was taking way more time during training, each epoch was taking almost 32 minutes while  “distilbert” model was taking about 24 minutes. So, we selected “distilbert” model afterward for further experimentation.

# Introducing Achitectural Changes to above model

To introduce meaningful architectural changes to my model for the NLI task, I consider several approaches, such as:

Integrating Sentence-Level Embeddings:Using pre-trained sentence embeddings to enhance the input representation.

Using Attention Mechanisms:Adding an attention layer to focus on important parts of the input.

Combining Multiple Pre-trained Models:
Using two different models and combining their outputs.

below is the implementation of a model that integrates sentence-level embeddings and uses an attention mechanism. We'll use sentence embeddings from a pre-trained model like sentence-transformers and an attention mechanism to enhance the model's performance.

# Model Output:

The model processes the input tokens (input_ids) and their attention masks (attention_mask).
It extracts the [CLS] token representation from DistilBERT, which is typically used for classification tasks.
Projection Layer:

The [CLS] token output (768 dimensions) is projected down to 384 dimensions to match the size of the sentence embeddings.
Sentence Embeddings:

The input sentences are decoded from token IDs back into text.
Sentence embeddings are generated using the SentenceTransformer model. These embeddings capture the overall meaning of the sentences.
Attention Mechanism:

The projected output from DistilBERT and the sentence embeddings are passed through a multi-head attention layer.
The attention mechanism helps the model focus on different parts of the sentence embeddings, potentially capturing more useful information.
Combining Outputs:

The outputs from the attention layer and the projected DistilBERT output are concatenated.
The combined output (768 dimensions) is passed through the final classifier layer to produce logits for classification.
Loss Calculation:

If labels are provided, the model calculates the cross-entropy loss between the predicted logits and the true labels.

In [ ]:
# Access the training data
device = "cuda" if torch.cuda.is_available() else "cpu"
# Model Parameters
language_model_name = "distilbert-base-uncased"
sentence_embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
# Training Arguments
batch_size = 16
learning_rate = 1e-4
# learning_rate = 1e-5
weight_decay = 0.001
# weight_decay = 0.01
epochs = 1
set_seed(42)
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(language_model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)#Pads sequences to the same length in a batch.
sentence_embedding_model = SentenceTransformer(sentence_embedding_model_name)

c:\Users\shahk\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Custom model with sentence embeddings and attention
class CustomModel(nn.Module):
    def __init__(self, base_model_name, sentence_embedding_model, num_labels):
        super(CustomModel, self).__init__()
        self.base_model = DistilBertForSequenceClassification.from_pretrained(
            base_model_name,
            num_labels=num_labels
        )
        self.sentence_embedding_model = sentence_embedding_model
        self.projection = nn.Linear(self.base_model.config.hidden_size, 384)  # Project DistilBERT output to match sentence embeddings
        self.attention = nn.MultiheadAttention(embed_dim=384, num_heads=4)
        self.dropout = nn.Dropout(0.3)  # Dropout for regularization
        self.classifier = nn.Linear(384 + 384, num_labels)  # Combine both outputs

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Get DistilBERT outputs
        outputs = self.base_model.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0]  # Take the [CLS] token representation

        # Project DistilBERT output
        projected_output = self.projection(pooled_output)

        # Get sentence embeddings
        with torch.no_grad():
            sentences = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
            sentence_embeddings = torch.tensor(self.sentence_embedding_model.encode(sentences)).to(input_ids.device)

        # Apply attention mechanism
        attn_output, _ = self.attention(projected_output.unsqueeze(0), sentence_embeddings.unsqueeze(0), sentence_embeddings.unsqueeze(0))
        attn_output = attn_output.squeeze(0)

        # Combine outputs and apply dropout
        combined_output = torch.cat((projected_output, attn_output), dim=1)
        combined_output = self.dropout(combined_output)
        logits = self.classifier(combined_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.base_model.num_labels), labels.view(-1))

        return {'loss': loss, 'logits': logits}


In [ ]:
# Initialize the custom model
model = CustomModel(language_model_name, sentence_embedding_model, num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",                    # output directory [Mandatory]
    num_train_epochs=2,                      # total number of training epochs
    per_device_train_batch_size=batch_size,       # batch size per device during training
    warmup_steps=500,                             # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                    # strength of weight decay
    save_strategy="no",
    learning_rate=learning_rate                   # learning rate
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)
trainer.train()

  0%|          | 0/6386 [00:00<?, ?it/s]

c:\Users\shahk\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.7016, 'grad_norm': 10.575435638427734, 'learning_rate': 0.0001, 'epoch': 0.16}
{'loss': 0.5622, 'grad_norm': 2.985837936401367, 'learning_rate': 9.150526673462454e-05, 'epoch': 0.31}
{'loss': 0.5176, 'grad_norm': 4.099152088165283, 'learning_rate': 8.301053346924907e-05, 'epoch': 0.47}
{'loss': 0.4995, 'grad_norm': 6.388758659362793, 'learning_rate': 7.45158002038736e-05, 'epoch': 0.63}
{'loss': 0.4755, 'grad_norm': 5.958062171936035, 'learning_rate': 6.602106693849813e-05, 'epoch': 0.78}
{'loss': 0.4857, 'grad_norm': 5.321105480194092, 'learning_rate': 5.7526333673122665e-05, 'epoch': 0.94}
{'loss': 0.3936, 'grad_norm': 2.9125046730041504, 'learning_rate': 4.90316004077472e-05, 'epoch': 1.1}
{'loss': 0.3617, 'grad_norm': 1.0651590824127197, 'learning_rate': 4.0536867142371735e-05, 'epoch': 1.25}
{'loss': 0.3481, 'grad_norm': 2.4638373851776123, 'learning_rate': 3.204213387699626e-05, 'epoch': 1.41}
{'loss': 0.3569, 'grad_norm': 5.123266696929932, 'learning_rate': 2.35474006

TrainOutput(global_step=6386, training_loss=0.4373035735558314, metrics={'train_runtime': 1970.1387, 'train_samples_per_second': 51.86, 'train_steps_per_second': 3.241, 'total_flos': 0.0, 'train_loss': 0.4373035735558314, 'epoch': 2.0})

Evaluating on standard testset

In [ ]:
results_standard = trainer.evaluate(tokenized_datasets['test'])
print(f"Standard Test Set Results: {results_standard}")

  0%|          | 0/286 [00:00<?, ?it/s]

Standard Test Set Results: {'eval_loss': 0.8944422006607056, 'eval_accuracy': 0.7031045037166593, 'eval_precision': 0.7019789096954616, 'eval_recall': 0.7031045037166593, 'eval_f1': 0.6950522346776756, 'eval_roc_auc': 0.8559390820358278, 'eval_hamming_loss': 0.2968954962833406, 'eval_runtime': 18.9191, 'eval_samples_per_second': 120.883, 'eval_steps_per_second': 15.117, 'epoch': 2.0}


In [ ]:
# Evaluate on the adversarial test set
results_adversarial = trainer.evaluate(adversarial_tokenized)
print(f"Adversarial Test Set Results: {results_adversarial}")

  0%|          | 0/43 [00:00<?, ?it/s]

Adversarial Test Set Results: {'eval_test_loss': 1.5860499143600464, 'eval_test_accuracy': 0.49851632047477745, 'eval_test_precision': 0.5114163504186483, 'eval_test_recall': 0.49851632047477745, 'eval_test_f1': 0.5004167917663728, 'eval_test_roc_auc': 0.6528492691464999, 'eval_test_hamming_loss': 0.5014836795252225, 'eval_test_runtime': 3.0677, 'eval_test_samples_per_second': 109.853, 'eval_test_steps_per_second': 14.017, 'epoch': 2.0}


<h2>Combining adverserial and Original dataset

In below cell it is being assumed that the adverserial dataset has been stored in the mentioned path and being loaded. refer the the data augmentation notebook

In [ ]:
from datasets import load_from_disk
Adverserial_Dataset = load_from_disk('adverserial_dataset')

In [ ]:
from datasets import Dataset, DatasetDict, concatenate_datasets

# Combine the train set of the first dataset with the second dataset
combined_train_dataset = concatenate_datasets([dataset['train'], Adverserial_Dataset])

# Shuffle the combined dataset
shuffled_combined_train_dataset = combined_train_dataset.shuffle(seed=42)

# Replace the train set in the first dataset with the shuffled combined dataset
combined_dataset_dict = DatasetDict({
    'train': shuffled_combined_train_dataset,
    'validation': dataset['validation'],
    'test': dataset['test']
})

print(combined_dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 66086
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2288
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
        num_rows: 2287
    })
})


# Training another model with Adverserial training dataset

In [ ]:
# Modify the configuration to set a custom dropout rate
num_labels=3
language_model_name = "distilbert-base-uncased"
config = DistilBertConfig.from_pretrained(language_model_name)
config.hidden_dropout_prob = 0.3  # Set your custom dropout rate here
config.num_labels = num_labels
config.output_attentions = False
config.output_hidden_states = False
# Initialize the model
# Load model and tokenizer with modified configuration
model = AutoModelForSequenceClassification.from_pretrained(
    language_model_name,
    config=config,
    ignore_mismatched_sizes=True
)
# model = AutoModelForSequenceClassification.from_pretrained(language_model_name,
#                                                                    config=config,
#                                                                    ignore_mismatched_sizes=True,
#                                                                    output_attentions=False,
#                                                                    output_hidden_states=False,
#                                                                    num_labels=3) # number of the classes

tokenizer = AutoTokenizer.from_pretrained(language_model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

c:\Users\shahk\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_datasets = combined_dataset_dict.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.map(encode_labels)

In [ ]:
batch_size = 16
# optim
learning_rate = 1e-4
weight_decay = 0.001 # we could use e.g. 0.01 in case of very low and very high amount of data for regularization
# training
epochs = 1
set_seed(42)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",                    # output directory [Mandatory]
    num_train_epochs=epochs,                      # total number of training epochs
    per_device_train_batch_size=batch_size,       # batch size per device during training
    warmup_steps=500,                             # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                    # strength of weight decay
    save_strategy="no",
    learning_rate=learning_rate                   # learning rate
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics
)
trainer.train()

  0%|          | 0/4131 [00:00<?, ?it/s]

{'loss': 0.7201, 'grad_norm': 3.4628992080688477, 'learning_rate': 0.0001, 'epoch': 0.12}
{'loss': 0.5949, 'grad_norm': 3.975820541381836, 'learning_rate': 8.622968879096668e-05, 'epoch': 0.24}
{'loss': 0.541, 'grad_norm': 3.5909810066223145, 'learning_rate': 7.245937758193335e-05, 'epoch': 0.36}
{'loss': 0.5128, 'grad_norm': 4.571284294128418, 'learning_rate': 5.868906637290003e-05, 'epoch': 0.48}
{'loss': 0.4872, 'grad_norm': 4.745654106140137, 'learning_rate': 4.491875516386671e-05, 'epoch': 0.61}
{'loss': 0.4619, 'grad_norm': 3.651492118835449, 'learning_rate': 3.114844395483338e-05, 'epoch': 0.73}
{'loss': 0.446, 'grad_norm': 26.286725997924805, 'learning_rate': 1.7378132745800055e-05, 'epoch': 0.85}
{'loss': 0.4247, 'grad_norm': 5.264534950256348, 'learning_rate': 3.607821536766731e-06, 'epoch': 0.97}
{'train_runtime': 1118.2704, 'train_samples_per_second': 59.097, 'train_steps_per_second': 3.694, 'train_loss': 0.5198253843057706, 'epoch': 1.0}


TrainOutput(global_step=4131, training_loss=0.5198253843057706, metrics={'train_runtime': 1118.2704, 'train_samples_per_second': 59.097, 'train_steps_per_second': 3.694, 'total_flos': 2188599156758016.0, 'train_loss': 0.5198253843057706, 'epoch': 1.0})

Evaluating on standard testset

In [ ]:
results_standard = trainer.evaluate(tokenized_datasets['test'])
print(f"Standard Test Set Results: {results_standard}")

  0%|          | 0/286 [00:00<?, ?it/s]

Standard Test Set Results: {'eval_loss': 0.8289945721626282, 'eval_accuracy': 0.6965456930476607, 'eval_precision': 0.7023926852096799, 'eval_recall': 0.6965456930476607, 'eval_f1': 0.6848846509973214, 'eval_roc_auc': 0.8562204991042014, 'eval_hamming_loss': 0.3034543069523393, 'eval_runtime': 13.3017, 'eval_samples_per_second': 171.933, 'eval_steps_per_second': 21.501, 'epoch': 1.0}


Evaluating on Adverserial Dataset

In [ ]:
adversarial_tokenized = adversarial_test_dataset.map(preprocess_function, batched=True)
adversarial_tokenized = adversarial_tokenized.map(encode_labels)
# Evaluate on the adversarial test set
results_adversarial = trainer.evaluate(adversarial_tokenized)
print(f"Adversarial Test Set Results: {results_adversarial}")

  0%|          | 0/43 [00:00<?, ?it/s]

Adversarial Test Set Results: {'eval_test_loss': 1.4084587097167969, 'eval_test_accuracy': 0.5044510385756676, 'eval_test_precision': 0.5360963355484721, 'eval_test_recall': 0.5044510385756676, 'eval_test_f1': 0.5057083872521098, 'eval_test_roc_auc': 0.6579466121552624, 'eval_test_hamming_loss': 0.49554896142433236, 'eval_test_runtime': 2.056, 'eval_test_samples_per_second': 163.914, 'eval_test_steps_per_second': 20.915, 'epoch': 1.0}
